In [ ]:
import sqlite3
import pandas as pd
csv = r"Test.csv"
pd.set_option('future.no_silent_downcasting', True)

#Export as csv, do regression in R, 95% CI

In [7]:
def check_col(name, data):
    types = []
    count = {}
    for val in data[name]:
        if val not in types:
            types.append(val)
            count[val] = 1
        else:
            count[val] += 1
    return types

In [ ]:
df = pd.read_csv(csv)
df["MultipleLines"] = df["MultipleLines"].replace({"No": 0.7, "No phone service": 0})
df = df.replace({"Yes": 1, "No": 0, "Male": 1, "Female": 0})
types = check_col("Contract", df)
df = df.replace({types[0]: 1/24, types[1]: 1/2, types[2]: 1})
types = check_col("InternetService", df)
df = df.replace({types[0]: 1, types[1]: 1})
df


,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,0,0,1,0,1,0,No phone service,1,0,...,0,0,0,0,0.041667,1,Electronic check,29.85,29.85,0
1,5575-GNVDE,1,0,0,0,34,1,0.7,1,1,...,1,0,0,0,0.5,0,Mailed check,56.95,1889.5,0
2,3668-QPYBK,1,0,0,0,2,1,0.7,1,1,...,0,0,0,0,0.041667,1,Mailed check,53.85,108.15,1
3,7795-CFOCW,1,0,0,0,45,0,No phone service,1,1,...,1,1,0,0,0.5,0,Bank transfer (automatic),42.30,1840.75,0
4,9237-HQITU,0,0,0,0,2,1,0.7,1,0,...,0,0,0,0,0.041667,1,Electronic check,70.70,151.65,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7038,6840-RESVB,1,0,1,1,24,1,1,1,1,...,1,1,1,1,0.5,1,Mailed check,84.80,1990.5,0
7039,2234-XADUH,0,0,1,1,72,1,1,1,0,...,1,0,1,1,0.5,1,Credit card (automatic),103.20,7362.9,0
7040,4801-JZAZL,0,0,1,1,11,0,No phone service,1,1,...,0,0,0,0,0.041667,1,Electronic check,29.60,346.45,0
7041,8361-LTMKD,1,1,1,0,4,1,1,1,0,...,0,0,0,0,0.041667,1,Mailed check,74.40,306.6,1
